# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [10]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [11]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [12]:
data.head()


,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [13]:
data_group=data.groupby(['CustomerID','ProductName']).agg({'Quantity':np.sum})
# data_group.Quantity = data_group.Quantity.sum()
data_group.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [14]:
data_pivot=pd.pivot_table(data_group, values='Quantity', index=['ProductName'],columns=['CustomerID'],fill_value=0)
data_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [15]:
squareform(pdist(data_pivot, 'euclidean'))

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [16]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'euclidean'))), 
                         index=data_pivot.columns, columns=data_pivot.columns)

similarities = distances[33].sort_values(ascending=False)[0:5]
similarities

CustomerID
33      1.000000
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [17]:
recommendations = data_group.copy()

list_series=[recommendations.loc[name] for name, score in dict(similarities).items()]

df_step5 = pd.concat(list_series)
df_step5.head()

,Quantity
ProductName,
Apricots - Dried,1
Assorted Desserts,1
Bandage - Flexible Neon,1
"Bar Mix - Pina Colada, 355 Ml",1
"Beans - Kidney, Canned",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [18]:
prod_quantity=df_step5.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
prod_quantity.head()

,Quantity
ProductName,
"Pepper - Black, Whole",4
Butter - Unsalted,3
Grouper - Fresh,3
"Veal - Inside, Choice",3
Veal - Sweetbread,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [19]:
data_pivot[data_pivot[33]==0].merge(prod_quantity, left_on='ProductName', right_on='ProductName').sort_values(by='Quantity',ascending=False)[[33,'Quantity']].head()

,33,Quantity
ProductName,,
Soup - Campbells Bean Medley,0,3
Butter - Unsalted,0,3
Lamb - Ground,0,2
"Mushrooms - Black, Dried",0,2
Chocolate - Dark,0,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [22]:
dictionary_step8={}

def generate_product(metric):
    distances = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, metric))), 
                         index=data_pivot.columns, columns=data_pivot.columns)
    dictionary_step8={}
    list_uniqueCustomers=list(data['CustomerID'].values)

    for i in list_uniqueCustomers:
        similarities_2 = distances[i].sort_values(ascending=False)[0:5]
        recommendations_2 = data_group.copy()
        list_series_2=[recommendations_2.loc[name] for name, score in dict(similarities_2).items()]
        df_step5_2 = pd.concat(list_series_2)
        prod_quantity_2=df_step5_2.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
        dictionary_step8[i]=(data_pivot[data_pivot[i]==0].merge(prod_quantity_2, left_on='ProductName', right_on='ProductName')[[i,'Quantity']].sort_values(by='Quantity',ascending=False).head()).index
    return dictionary_step8

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [23]:
pd.DataFrame.from_dict(generate_product('euclidean'))

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Chicken - Soup Base,Tuna - Salad Premix,Bread - Italian Corn Meal Poly,Flavouring - Orange,Lettuce - Frisee,Cocoa Butter,Juice - Lime,Bread - French Baquette,Hersey Shakes,Quiche Assorted,...,Beef - Top Sirloin,"Pasta - Penne, Rigate, Dry",Milk - 2%,Rum - Mount Gay Eclipes,Chips Potato Salt Vinegar 43g,"Pasta - Penne, Rigate, Dry",Veal - Inside,Olives - Kalamata,"Veal - Brisket, Provimi,bnls",Wine - Blue Nun Qualitatswein
1,Lime Cordial - Roses,Chicken - Soup Base,Grouper - Fresh,Bread - Raisin Walnut Oval,Beef - Inside Round,"Cheese - Boursin, Garlic / Herbs",Tomatoes Tear Drop,Bandage - Flexible Neon,Veal - Sweetbread,Cheese - Mozzarella,...,Grouper - Fresh,Rambutan,Banana Turning,Muffin Mix - Blueberry,Fenngreek Seed,Browning Caramel Glace,Bread Crumbs - Japanese Style,Wine - White Cab Sauv.on,Beef - Ground Medium,Beef Wellington
2,Wine - Prosecco Valdobiaddene,Tahini Paste,Sherry - Dry,Banana Turning,Pop Shoppe Cream Soda,Cheese - Mix,Pomello,Thermometer Digital,Soup Knorr Chili With Beans,"Lemonade - Natural, 591 Ml",...,Wine - Charddonnay Errazuriz,Cheese Cloth No 100,Bay Leaf,Cheese - Mix,Bay Leaf,"Garlic - Primerba, Paste",Wonton Wrappers,Bread - Calabrese Baguette,Scallops - 10/20,Knife Plastic - White
3,Pasta - Orecchiette,Isomalt,Chocolate - Feathers,Squid - Tubes / Tenticles 10/20,Sausage - Liver,Sauce - Hollandaise,Skirt - 29 Foot,Wine - Hardys Bankside Shiraz,Soupfoamcont12oz 112con,Mangoes,...,Lime Cordial - Roses,Soup Knorr Chili With Beans,Beer - Labatt Blue,Scallops 60/80 Iqf,Isomalt,"Cheese - Boursin, Garlic / Herbs","Placemat - Scallop, White",Cheese - Wine,"Rosemary - Primerba, Paste",Table Cloth 54x72 White
4,Dc Hikiage Hira Huba,Wine - Valpolicella Masi,Puree - Mocha,Steam Pan - Half Size Deep,Loquat,Dc Hikiage Hira Huba,Soup - Campbells Tomato Ravioli,"Lentils - Red, Dry",Blueberries,"Rosemary - Primerba, Paste",...,Beef - Striploin Aa,Phyllo Dough,Wine - Vidal Icewine Magnotta,Cake - Box Window 10x10x2.5,Veal - Sweetbread,Beets - Mini Golden,Wasabi Powder,Wine - Two Oceans Cabernet,Tea - Decaf Lipton,Sherry - Dry


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [24]:
list_metric=['correlation','cityblock','cosine','jaccard']
for i in list_metric:
    display(pd.DataFrame.from_dict(generate_product(i)))

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,"Soup - Campbells, Cream Of",Grenadine,"Bread - Roll, Soft White Round",Olives - Stuffed,"Tart Shells - Sweet, 4",Pastry - Cheese Baked Scones,"Nut - Chestnuts, Whole","Veal - Brisket, Provimi,bnls",Vaccum Bag 10x13,Banana Turning,...,Wasabi Powder,Beef - Inside Round,"Veal - Inside, Choice",Foam Cup 6 Oz,Wine - Blue Nun Qualitatswein,Soup Knorr Chili With Beans,"Salsify, Organic","Potatoes - Instant, Mashed",Campari,"Wine - Red, Colio Cabernet"
1,Ecolab - Solid Fusion,Mangoes,Truffle Cups - Brown,Wine - Charddonnay Errazuriz,"Appetizer - Mini Egg Roll, Shrimp",Pastry - Choclate Baked,"Beef - Tenderlion, Center Cut",Cheese Cloth No 100,Pastry - Choclate Baked,"Pork - Loin, Bone - In",...,Gatorade - Xfactor Berry,Grenadine,Beer - Labatt Blue,Oil - Shortening - All - Purpose,Wine - White Cab Sauv.on,Barramundi,"Hickory Smoke, Liquid","Chestnuts - Whole,canned",Bread Fig And Almond,"Nut - Pistachio, Shelled"
2,Tofu - Firm,Lettuce - Frisee,Juice - V8 Splash,Assorted Desserts,Chocolate - Dark,Bouq All Italian - Primerba,"Soup - Canadian Pea, Dry Mix",Pants Custom Dry Clean,Browning Caramel Glace,Wasabi Powder,...,Scallops - 10/20,Coffee - Hazelnut Cream,Wine - Vidal Icewine Magnotta,"Spoon - Soup, Plastic",Flour - Whole Wheat,Milk - 1%,Dc Hikiage Hira Huba,Tia Maria,"Veal - Inside, Choice",Creme De Banane - Marie
3,Juice - Apple Cider,Fondant - Icing,Anchovy Paste - 56 G Tube,Juice - Orange,Sauerkraut,Bread - Italian Roll With Herbs,Wine - Gato Negro Cabernet,Onions - Cippolini,Ecolab - Lime - A - Way 4/4 L,Nantuket Peach Orange,...,"Water, Tap","Oregano - Dry, Rubbed",Chocolate - Dark,Beef - Montreal Smoked Brisket,Beer - Rickards Red,"Soup - Campbells, Cream Of",Oil - Shortening - All - Purpose,Isomalt,Fondant - Icing,Napkin White - Starched
4,Langers - Ruby Red Grapfruit,Towels - Paper / Kraft,Beer - Sleemans Cream Ale,Soup - Campbells Tomato Ravioli,"Cup - 6oz, Foam",Thermometer Digital,Veal - Eye Of Round,"Bread - Roll, Canadian Dinner",Island Oasis - Mango Daiquiri,Spinach - Baby,...,Wine - Blue Nun Qualitatswein,Wine - Fume Blanc Fetzer,Bay Leaf,Cookie Dough - Double,Wine - Fume Blanc Fetzer,Beef - Texas Style Burger,Banana - Leaves,Veal - Eye Of Round,"Salmon - Atlantic, Skin On","Appetizer - Mini Egg Roll, Shrimp"


,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Knife Plastic - White,Veal - Sweetbread,Butter - Unsalted,"Cheese - Boursin, Garlic / Herbs",Knife Plastic - White,"Nut - Chestnuts, Whole",Juice - Lime,Pork - Kidney,Knife Plastic - White,Appetizer - Sausage Rolls,...,Foam Dinner Plate,Soup - Campbells Bean Medley,Cheese - Parmesan Grated,Wine - Blue Nun Qualitatswein,"Cheese - Boursin, Garlic / Herbs",Curry Paste - Madras,Pork - Kidney,Sobe - Tropical Energy,Pork - Hock And Feet Attached,Pork - Kidney
1,Bay Leaf,Tia Maria,Wine - Ej Gallo Sierra Valley,Juice - Lime,Butter - Unsalted,Wine - Blue Nun Qualitatswein,Tomatoes Tear Drop,"Cheese - Boursin, Garlic / Herbs",Butter - Unsalted,"Peas - Pigeon, Dry",...,Muffin - Carrot Individual Wrap,Yeast Dry - Fermipan,Quiche Assorted,"Wine - Magnotta, Merlot Sr Vqa",Ice Cream Bar - Hageen Daz To,Fond - Neutral,Muffin - Carrot Individual Wrap,Tomatoes Tear Drop,"Pepper - Paprika, Hungarian","Cheese - Boursin, Garlic / Herbs"
2,Soup - Campbells Bean Medley,Juice - Lime,Muffin Batt - Blueberry Passion,Soup - Campbells Tomato Ravioli,Tia Maria,Olive - Spread Tapenade,Pants Custom Dry Clean,Lime Cordial - Roses,"Pepper - Black, Whole",Foam Dinner Plate,...,Halibut - Steaks,Apricots - Dried,Sugar - Fine,Sherry - Dry,"Pepper - White, Ground",Crush - Cream Soda,"Cheese - Boursin, Garlic / Herbs",Knife Plastic - White,"Pork - Bacon, Double Smoked",Muffin - Carrot Individual Wrap
3,Sausage - Liver,"Sole - Dover, Whole, Fresh",Lamb - Ground,Ice Cream Bar - Hageen Daz To,Beef - Montreal Smoked Brisket,"Oil - Shortening,liqud, Fry",Cheese - Taleggio D.o.p.,Scampi Tail,Olive - Spread Tapenade,Fondant - Icing,...,Soup - Campbells Tomato Ravioli,"Pasta - Penne, Rigate, Dry",Crush - Cream Soda,"Water, Tap",Soup - Campbells Tomato Ravioli,Knife Plastic - White,Knife Plastic - White,Crush - Cream Soda,Juice - Lime,"Pasta - Detalini, White, Fresh"
4,Butter - Unsalted,Table Cloth 81x81 White,"Cheese - Boursin, Garlic / Herbs",Squid - Tubes / Tenticles 10/20,Muffin - Carrot Individual Wrap,Oil - Shortening - All - Purpose,Butter - Unsalted,Cheese - Parmesan Cubes,Ezy Change Mophandle,Halibut - Steaks,...,Pork - Kidney,Cheese Cloth No 100,Tia Maria,Olives - Stuffed,Sage - Ground,Muffin - Carrot Individual Wrap,Ketchup - Tomato,"Spoon - Soup, Plastic","Pepper - White, Ground",Knife Plastic - White


,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,"Soup - Campbells, Cream Of",Fondant - Icing,"Bread - Roll, Soft White Round",Wine - Charddonnay Errazuriz,"Tart Shells - Sweet, 4",Bouq All Italian - Primerba,Wine - Gato Negro Cabernet,"Bread - Roll, Canadian Dinner",Vaccum Bag 10x13,Spinach - Baby,...,Wasabi Powder,Potatoes - Idaho 100 Count,Bay Leaf,Foam Cup 6 Oz,"Bar Mix - Pina Colada, 355 Ml",Soup Knorr Chili With Beans,"Salsify, Organic",Bread - Calabrese Baguette,Fondant - Icing,"Wine - Red, Colio Cabernet"
1,Ecolab - Solid Fusion,Lettuce - Frisee,Truffle Cups - Brown,Assorted Desserts,"Appetizer - Mini Egg Roll, Shrimp",Onions - Vidalia,"Mushroom - Trumpet, Dry",Butter - Unsalted,Pastry - Choclate Baked,Foam Cup 6 Oz,...,Gatorade - Xfactor Berry,Macaroons - Two Bite Choc,Banana Turning,Oil - Shortening - All - Purpose,Pastry - Raisin Muffin - Mini,Barramundi,"Hickory Smoke, Liquid",Foam Dinner Plate,Nantucket - Pomegranate Pear,Creme De Banane - Marie
2,Tofu - Firm,Wine - Redchard Merritt,Juice - V8 Splash,Beef - Texas Style Burger,Chocolate - Dark,Scallops - Live In Shell,Mayonnaise - Individual Pkg,Smoked Paprika,Browning Caramel Glace,Sprouts - Baby Pea Tendrils,...,Scallops - 10/20,Raspberries - Fresh,Milk - 2%,"Spoon - Soup, Plastic",Bread - Italian Corn Meal Poly,Milk - 1%,Dc Hikiage Hira Huba,Cheese - Parmesan Grated,Longos - Chicken Wings,Napkin White - Starched
3,Juice - Apple Cider,"Oregano - Dry, Rubbed",Anchovy Paste - 56 G Tube,Juice - Orange,Sauerkraut,Pastry - Choclate Baked,"Soup - Canadian Pea, Dry Mix",Snapple Lemon Tea,Ecolab - Lime - A - Way 4/4 L,Smirnoff Green Apple Twist,...,"Water, Tap",Scallops - Live In Shell,Sauce - Demi Glace,Beef - Montreal Smoked Brisket,Whmis - Spray Bottle Trigger,"Soup - Campbells, Cream Of",Oil - Shortening - All - Purpose,Olives - Kalamata,"Salmon - Atlantic, Skin On","Appetizer - Mini Egg Roll, Shrimp"
4,Langers - Ruby Red Grapfruit,Towels - Paper / Kraft,Beer - Sleemans Cream Ale,Yogurt - French Vanilla,"Cup - 6oz, Foam",Bread - Italian Roll With Herbs,Olives - Stuffed,Soupcontfoam16oz 116con,Island Oasis - Mango Daiquiri,Apricots Fresh,...,Wine - Blue Nun Qualitatswein,Garlic - Peeled,Chocolate - Dark,Cookie Dough - Double,Pecan Raisin - Tarts,Beef - Texas Style Burger,Banana - Leaves,Bread - Rye,Garbag Bags - Black,"Sauce - Gravy, Au Jus, Mix"


,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Ecolab - Mikroklene 4/4 L,Garbag Bags - Black,Rosemary - Dry,Wine - Hardys Bankside Shiraz,Muffin Batt - Blueberry Passion,Pork - Inside,Ocean Spray - Kiwi Strawberry,"Placemat - Scallop, White",Zucchini - Yellow,Muffin - Carrot Individual Wrap,...,Pie Filling - Cherry,French Pastry - Mini Chocolate,Onion Powder,"Pork - Back, Short Cut, Boneless",Scallops 60/80 Iqf,Flavouring - Orange,Berry Brulee,Lettuce - Treviso,"Soup - Canadian Pea, Dry Mix",Cumin - Whole
1,Flavouring - Orange,Mayonnaise - Individual Pkg,Pasta - Angel Hair,Cookie Dough - Double,Cocoa Butter,Sprouts - Alfalfa,Veal - Osso Bucco,Broom - Corn,Cinnamon Buns Sticky,Crush - Cream Soda,...,Lettuce - Treviso,"Nut - Pistachio, Shelled",Bouq All Italian - Primerba,Mussels - Frozen,Dc Hikiage Hira Huba,"Bread - Roll, Canadian Dinner",Wine - Vineland Estate Semi - Dry,Scampi Tail,Rice - Jasmine Sented,Watercress
2,Appetizer - Sausage Rolls,Water - Aquafina Vitamin,Sponge Cake Mix - Chocolate,Soup - Campbells Bean Medley,Pate - Cognac,Cheese - Victor Et Berthold,Beets - Mini Golden,Beef - Top Sirloin,"Beans - Kidney, Red Dry",Cheese - Mix,...,Cup - Translucent 7 Oz Clear,Cocoa Butter,Tomato - Tricolor Cherry,"Cheese - Boursin, Garlic / Herbs",Tuna - Salad Premix,Juice - Happy Planet,Muffin Batt - Blueberry Passion,Pastry - Cheese Baked Scones,Cheese - Victor Et Berthold,Veal - Inside
3,Mustard - Seed,Beef Ground Medium,"Wine - Red, Colio Cabernet",Cookies Cereal Nut,Sword Pick Asst,Lettuce - Frisee,Cheese - Victor Et Berthold,Chips Potato Salt Vinegar 43g,"Cheese - Brie,danish",Tilapia - Fillets,...,Table Cloth - 53x69 Colour,Juice - Lime,Bread - Hot Dog Buns,Longos - Grilled Chicken With,Tahini Paste,Squid U5 - Thailand,"Ice - Clear, 300 Lb For Carving",Hot Chocolate - Individual,"Cup - 6oz, Foam",Sprouts - Baby Pea Tendrils
4,Cornflakes,Bread - Raisin Walnut Oval,Ketchup - Tomato,Cookie - Dough Variety,Vinegar - Tarragon,"Pork - Loin, Bone - In","Wine - Magnotta, Merlot Sr Vqa",Sausage - Breakfast,Rice - Jasmine Sented,"Wine - White, Colubia Cresh",...,Wine - Vineland Estate Semi - Dry,Cake - Mini Cheesecake,"Bread - Roll, Canadian Dinner",Ketchup - Tomato,Cocktail Napkin Blue,Lettuce - Frisee,Garbage Bags - Clear,"Pepper - Black, Whole","Wine - Cahors Ac 2000, Clos",Spice - Peppercorn Melange
